## Loading Modules

In [1]:
import os
import numpy as np
import pandas as pd

from utils import *
from calibration import *

from synapticTrack.io import BeamDataIOManager
from synapticTrack.lattice.track_elements import *
from synapticTrack.lattice.track_lattice import Lattice
from synapticTrack.track.run_track import *

## Run Simulations

In [2]:
import time
t0 = time.time()

# Load LEBT lattice
elements_list, stms = load_LEBT_lattice()

# ouput files and directories
output_dir_list = ['run.wire_scanner1', 'run.allison_scanner', 'run.wire_scanner2', 'run.wire_scanner3', 'run.wire_scanner4', 'run.ends']

final_output_files = ['coord_wire_scanner1.out', 'coord_allison_scanner.out', 'coord_wire_scanner2.out', 'coord_wire_scanner3.out', 
                      'coord_wire_scanner4.out', 'coord_ends.out']

fh = [5.0, 5.0, 5.0, 5.0, 5.0]
fv = [5.0, 5.0, 5.0, 5.0, 5.0]

stm_scale_x = np.load("stm_scale_x.npy")
stm_scale_y = np.load("stm_scale_y.npy")

sim_centroid_x = np.zeros((4, 5, 3), dtype=float)
sim_centroid_y = np.zeros((4, 5, 3), dtype=float)

count = 0

for i, stm in zip(range(0, 5), ['stm0', 'stm1', 'stm2', 'stm3', 'stm4']):
    for j, dir_end, kick in zip([0, 1, 2], ['p', 'm', 'n'], [1.1, -1.1, 0.0]):
        for k, direction in zip([0, 1], ['x', 'y']):
            fhkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            fvkick = [0.0, 0.0, 0.0, 0.0, 0.0]
            if direction == 'x':
                fhkick[i] = kick * stm_scale_x[i]
            elif direction == 'y':
                fvkick[i] = kick * stm_scale_y[i]

            final_output_dir = "stm." + str(i) + direction + dir_end
            os.makedirs(final_output_dir, exist_ok=True)

            set_stm_strengths(stms, fhkick, fvkick)
            #elements_list = construct_lattice_segments()

            run_track_steps(elements_list, output_dir_list)
            copy_step_output_files(final_output_dir, output_dir_list, final_output_files)
            copy_track_output_files(final_output_dir, output_dir_list)

            centroid, rms = get_beam_data(final_output_dir, final_output_files)
            x_center = centroid['x'].apply(lambda x: float(f"{x: 11.8f}")).tolist()
            y_center = centroid['y'].apply(lambda x: float(f"{x: 11.8f}")).tolist()

            for l, ws in enumerate([0, 2, 3, 4]):
                if direction == 'x':
                    sim_centroid_x[l, i, j] = x_center[ws]
                elif direction == 'y':
                    sim_centroid_y[l, i, j] = y_center[ws]
            
            print (f"stm{i}", f"{kick: 4.1f}", direction, x_center, y_center)

ws_offset_x = np.load("ws_offset_x.npy")
ws_offset_y = np.load("ws_offset_y.npy")

sim_corr_x = sim_centroid_x - ws_offset_x[:, None, None]
sim_corr_y = sim_centroid_y - ws_offset_y[:, None, None]

np.save("sim_centroid_x_new.npy", sim_corr_x)
np.save("sim_centroid_y_new.npy", sim_corr_y)

t1 = time.time()
print (t1 - t0)

stm0  1.1 x [0.39794571, 0.74485467, -0.0563369, 0.44246276, 0.54862959, -0.05164461] [8.2e-07, -1.15e-06, 0.00057056, -0.00035957, -0.00020494, 5.304e-05]
stm0  1.1 y [0.04237972, -0.0263867, 0.09732468, -0.11729087, -0.23073674, -0.01096656] [0.39337034, 1.07466732, -1.003605, 0.37882183, 1.17047528, -0.07476509]
stm0 -1.1 x [-0.31318699, -0.79762951, 0.24969866, -0.67488302, -1.00637178, 0.02976455] [8.2e-07, -1.16e-06, 0.00056796, -0.00035695, -0.00020169, 4.997e-05]
stm0 -1.1 y [0.04237972, -0.02638618, 0.09732419, -0.11729557, -0.23074154, -0.01096496] [-0.39336883, -1.07467053, 1.00474613, -0.37953889, -1.17088416, 0.07486842]
stm0  0.0 x [0.04237972, -0.02638572, 0.09633771, -0.11552018, -0.22775217, -0.01094204] [8.2e-07, -7.5e-07, 0.00057037, -0.00035851, -0.0002041, 5.167e-05]
stm0  0.0 y [0.04237972, -0.02638572, 0.09633771, -0.11552018, -0.22775217, -0.01094204] [8.2e-07, -7.5e-07, 0.00057037, -0.00035851, -0.0002041, 5.167e-05]
stm1  1.1 x [0.04237972, 1.15756359, -1.3118

## Calibrate Steering Magnet

In [3]:
# ---------------------------------------------------------------
# Load numpy arrays (shape = (4 WS, 5 steerers, 3 kicks))
# kicks index order assumed to be: [ +1.1 mrad , -1.1 mrad , 0.0 ]
# ---------------------------------------------------------------
exp_x = np.load("exp_centroid_x.npy")   # shape (4,5,3)
exp_y = np.load("exp_centroid_y.npy")
sim_x = np.load("sim_centroid_x_new.npy")
sim_y = np.load("sim_centroid_y_new.npy")

print("Loaded centroid arrays with shape:", exp_x.shape)

WS = 4
STEERERS = 5
KICK_PLUS  = 0
KICK_MINUS = 1
KICK_ZERO  = 2

KICK_MAG = 1.1  # mrad nominal

Loaded centroid arrays with shape: (4, 5, 3)


In [4]:
# ---------------------------------------------------------------
# Check data
# ---------------------------------------------------------------

check_data(WS, STEERERS, exp_x, sim_x, exp_y, sim_y)

stm0: ws0:  0.521  0.665 -0.149 -0.047  0.815  1.455  0.108  0.668
stm0: ws1: -1.735 -1.135 -1.382 -0.829 -0.601 -0.625  1.500  1.383
stm0: ws2:  2.349  1.288  1.352  0.170  1.185  0.965  0.335  0.206
stm0: ws3:  2.712  0.806  1.061 -0.749 -1.764 -1.567 -4.023 -3.908

stm1: ws0:  0.000  0.309  0.000  0.309  0.000  1.062  0.000  1.062
stm1: ws1: -2.940 -2.391 -0.348  0.427 -0.420 -0.334  1.569  1.092
stm1: ws2:  2.973  1.792  0.823 -0.340  0.824  0.228  0.711  0.943
stm1: ws3:  4.902  2.666 -0.493 -2.616 -1.405 -0.988 -4.843 -4.487

stm2: ws0:  0.000  0.309  0.000  0.309  0.000  1.062  0.000  1.062
stm2: ws1: -1.841 -1.076 -1.542 -0.889  1.828  1.527 -0.535 -0.769
stm2: ws2:  1.431  0.220  2.424  1.239 -0.184 -0.301  1.534  1.472
stm2: ws3:  1.757 -0.439  2.700  0.497 -3.611 -3.411 -2.305 -2.064

stm3: ws0:  0.000  0.309  0.000  0.309  0.000  1.062  0.000  1.062
stm3: ws1: -1.177 -0.481 -2.202 -1.483  1.220  1.205 -0.393 -0.447
stm3: ws2:  1.590  0.447  2.218  1.013  0.721  0.499  0.956

In [5]:
# ---------------------------------------------------------------
# Compute Wire Scanner Offsets
# ---------------------------------------------------------------

ws_offset_x, ws_offset_y = compute_wire_scanner_offsets(WS, exp_x, sim_x, exp_y, sim_y, KICK_ZERO)

np.save("ws_offset_x_new.npy", ws_offset_x)
np.save("ws_offset_y_new.npy", ws_offset_y)

== Wire Scanner Offsets (simulation - experiment) ===
WS1: x_offset=+0.0000,   y_offset=+0.0000
WS2: x_offset=+0.0000,   y_offset=+0.0000
WS3: x_offset=+0.0000,   y_offset=+0.0000
WS4: x_offset=-0.0000,   y_offset=+0.0000


In [6]:
# ---------------------------------------------------------------
# Compute response matrices
# ---------------------------------------------------------------

from calibration import *
R_sim_x, R_exp_x = compute_response(WS, STEERERS, sim_x, exp_x, KICK_PLUS, KICK_MINUS, KICK_MAG)
R_sim_y, R_exp_y = compute_response(WS, STEERERS, sim_y, exp_y, KICK_PLUS, KICK_MINUS, KICK_MAG)

In [7]:
# ---------------------------------------------------------------
# Compute scale factors
# ---------------------------------------------------------------

scale_x = solve_scale_factors(STEERERS, R_sim_x, R_exp_x)
scale_y = solve_scale_factors(STEERERS, R_sim_y, R_exp_y)

np.save("stm_scale_x_new.npy", scale_x)
np.save("stm_scale_y_new.npy", scale_y)

print_scale_factors(STEERERS, scale_x, scale_y)

=== Steerer Scale Factors ===
Steerer 1:  s_x=1.0000,   s_y=1.0000
Steerer 2:  s_x=1.0000,   s_y=1.0000
Steerer 3:  s_x=1.0000,   s_y=1.0000
Steerer 4:  s_x=1.0000,   s_y=1.0000
Steerer 5:  s_x=1.0000,   s_y=1.0000


In [8]:
# ---------------------------------------------------------------
# Calibration summary
# ---------------------------------------------------------------
print_calibration_summary(WS, STEERERS, ws_offset_x, ws_offset_y, scale_x, scale_y)

=== SUMMARY ===
Wire Scanner Offsets (apply to simulation or embed into lattice):
  WS1:  dx=+0.0000,  dy=+0.0000
  WS2:  dx=+0.0000,  dy=+0.0000
  WS3:  dx=+0.0000,  dy=+0.0000
  WS4:  dx=-0.0000,  dy=+0.0000

Steerer Scale Factors (multiply TRACK FH/FV or FHkick/FVkick):
  SM1:  x_scale=1.0000,   y_scale=1.0000
  SM2:  x_scale=1.0000,   y_scale=1.0000
  SM3:  x_scale=1.0000,   y_scale=1.0000
  SM4:  x_scale=1.0000,   y_scale=1.0000
  SM5:  x_scale=1.0000,   y_scale=1.0000

Meaning:
- After applying WS offsets, simulation and experiment match at zero kick.
- After applying steerer scale factors, the ±1.1 mrad response matches experiment.

